In [1]:
import langchain
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2 

In [3]:
!pip install langchain_community

In [4]:
from langchain.chat_models import ChatOpenAI

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
key = os.getenv("OPENAI_API_KEY")

In [7]:
llm = ChatOpenAI(openai_api_key = key,model_name= 'gpt-3.5-turbo' , temperature = 1)

C:\Users\TECHNICAL\AppData\Local\Temp\ipykernel_16440\3665561483.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(openai_api_key = key,model_name= 'gpt-3.5-turbo' , temperature = 1)


In [ ]:
prompt1 = '''
You Are a helpful assistant, you are tasked to make an MCQ {exam} exam, you have to make it like this form\n{form}\n forming {questions} questions with level of difficulty of easy and the 2 last hard questions from this resource {text}
'''

In [13]:
temp1 = PromptTemplate(
    input_variable =['exam' , 'form' , 'questions' , 'text'],
    template = prompt1
)
chain1=LLMChain(llm=llm,prompt=temp1, output_key="quiz")

C:\Users\TECHNICAL\AppData\Local\Temp\ipykernel_16440\2353228756.py:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain1=LLMChain(llm=llm,prompt=temp1, output_key="quiz")


In [15]:
temp2 = PromptTemplate(
    input_variable = 'quiz',
    template = 'You are good question answerer for this quiz\n{quiz} give me the answers in bulleted list\n'
)
chain2=LLMChain(llm=llm,prompt=temp2, output_key="Answers") 

In [16]:
chain3=SequentialChain(chains=[chain1, chain2],
    input_variables = ['exam' , 'form' , 'questions' , 'text'],
    output_variables=["quiz","Answers"]
)

In [39]:
form = '''
1)question here:\n
a)first option\n
b)second option\n
c)third option\n
d)forth option\n
\n
2)question here:\n
a)first option\n
b)second option\n
c)third option\n
d)forth option\n
'''

In [47]:
with open(r"C:\University\NLP Course\mcqGen\text.txt" , 'r') as file:
    text = file.read()

In [50]:
r = chain3({"exam":"Machine learning" , "form":form ,"questions":10,"text":text })

In [54]:
with get_openai_callback() as cb:
    response = chain3(
        {
            "exam":"Machine learning" , 
            "form":form ,
            "questions":10,
            "text":text

        }


    )

In [56]:
response

{'exam': 'Machine learning',
 'form': '\n1)question here:\n\na)first option\n\nb)second option\n\nc)third option\n\nd)forth option\n\n\n\n2)question here:\n\na)first option\n\nb)second option\n\nc)third option\n\nd)forth option\n\n',
 'questions': 10,
 'text': "Machine Learning, often abbreviated as ML, is a subset of artificial intelligence (AI) that focuses on the development of computer algorithms that improve automatically through experience and by the use of data. In simpler terms, machine learning enables computers to learn from data and make decisions or predictions without being explicitly programmed to do so.\n\nAt its core, machine learning is all about creating and implementing algorithms that facilitate these decisions and predictions. These algorithms are designed to improve their performance over time, becoming more accurate and effective as they process more data.\n\nIn traditional programming, a computer follows a set of predefined instructions to perform a task. Howeve

In [57]:
for i in response['quiz'].split('\n'):
    print(i)

1) Which of the following best describes machine learning?

a) The development of programs that behave intelligently and mimic human intelligence
b) The use of algorithms that learn from data to make predictions
c) The study of multi-layered artificial neural networks inspired by the human brain
d) The process of giving a computer specific instructions on how to perform a task

2) What is the main goal of machine learning algorithms?

a) To follow a set of predefined instructions to perform a task
b) To uncover hidden patterns and provide valuable insights from data
c) To mimic human intelligence through a set of algorithms
d) To predict outcomes based on historical data

3) How does machine learning differ from traditional programming?

a) Machine learning algorithms do not require data to function
b) Traditional programming involves learning from examples, while machine learning does not
c) Machine learning allows computers to learn and improve over time without explicit programming


In [58]:
for i in response['Answers'].split('\n'):
    print(i)

- 1) b) The use of algorithms that learn from data to make predictions
- 2) d) To predict outcomes based on historical data
- 3) c) Machine learning allows computers to learn and improve over time without explicit programming
- 4) c) A subfield of machine learning that uses multi-layered artificial neural networks
- 5) a) To handle and make sense of large volumes of data
- 6) c) Healthcare
- 7) c) By improving efficiency and providing valuable insights from data
- 8) a) Recommendation systems
- 9) d) Learning from data and improving over time to perform previously manual tasks
- 10) d) 26%


In [59]:
print(cb)

Tokens Used: 2329
	Prompt Tokens: 1609
	Completion Tokens: 720
Successful Requests: 2
Total Cost (USD): $0.0038535
